# Dogtag PKI System Guide

## A Comprehensive Reference for Developers, Administrators, Auditors, and Security Professionals

**Version:** 11.10.0 (Development Branch: `tomcat-to-quarkus-migration`)
**Date:** February 2026
**License:** GPL-2.0 and LGPL-2.0

---

This notebook provides a complete technical reference for the Dogtag PKI system. Each section is written with a specific audience in mind:

| Audience | Sections of Interest |
|---|---|
| **End Users** | 1 (Overview), 3 (Subsystems), 9 (Certificate Profiles) |
| **Developers** | 2 (Architecture), 4 (Build System), 5 (Maven Modules), 6 (Code Structure), 11 (Quarkus Migration) |
| **System Administrators** | 3 (Subsystems), 7 (Deployment), 8 (Container/Docker), 10 (Server Management) |
| **Security Administrators** | 3 (Subsystems), 9 (Certificate Profiles), 12 (Access Control), 13 (Security Architecture) |
| **Auditors** | 12 (Access Control), 13 (Security Architecture), 14 (Audit & Compliance), 15 (CI/CD & Testing) |

---
# 1. System Overview
*Audience: All readers*

## What is Dogtag PKI?

Dogtag PKI is an **enterprise-class, open-source Certificate Authority (CA) system** developed by Red Hat. It provides a complete Public Key Infrastructure for managing the entire certificate lifecycle -- from issuance through renewal and revocation.

Organizations use Dogtag PKI to:
- Issue and manage X.509 digital certificates
- Provide secure key storage and recovery
- Validate certificate status via OCSP
- Support smart card (token) operations
- Automate certificate management via ACME and EST protocols
- Integrate with enterprise identity systems (FreeIPA/Red Hat IdM)

## Technology Stack

```
+------------------------------------------------------------------+
|                    Dogtag PKI Technology Stack                     |
+------------------------------------------------------------------+
| Layer              | Technology                                   |
|--------------------+----------------------------------------------|
| Language (Core)    | Java 17/21/25 (~683K lines, 3,494 files)     |
| Language (Mgmt)    | Python 3 (~60K lines, 189 files)             |
| Language (Native)  | C/C++ (109 source files)                     |
| App Server (Legacy)| Apache Tomcat 9.0 / 10.1                     |
| App Server (New)   | Quarkus 3.17.4 (migration in progress)       |
| Crypto Library     | Mozilla NSS / NSPR (FIPS 140-2 validated)    |
| Directory Service  | 389 Directory Server (LDAP)                  |
| REST Framework     | RESTEasy (JAX-RS)                            |
| Serialization      | Jackson (JSON/XML)                           |
| Build System       | CMake + Maven + RPM                          |
| Containers         | Podman / Docker (multi-stage Dockerfile)     |
| CI/CD              | GitHub Actions (187 workflow files)           |
+------------------------------------------------------------------+
```

---
# 2. Architecture Overview
*Audience: Developers, System Administrators*

## High-Level Architecture

```
                          +------------------+
                          |   End Users /     |
                          |   Applications    |
                          +--------+---------+
                                   |
                          HTTPS (8443/8080)
                                   |
                    +--------------v---------------+
                    |     Application Server       |
                    |   (Tomcat 9/10 or Quarkus)   |
                    +--------------+---------------+
                                   |
          +------------------------+------------------------+
          |            |           |          |             |
     +----v----+  +---v----+ +----v----+ +---v----+  +-----v-----+
     |   CA    |  |  KRA   | |  OCSP   | |  TKS   |  |    TPS    |
     | (Cert   |  | (Key   | | (Status | | (Token | |  (Token   |
     |  Auth)  |  | Recov) | | Validn) | |  Keys) |  |  Process) |
     +----+----+  +---+----+ +----+----+ +---+----+  +-----+-----+
          |            |           |          |             |
          +------------------------+------------------------+
                                   |
                    +--------------v---------------+
                    |    Shared Server Framework    |
                    |   (pki-server / pki-common)   |
                    +--------------+---------------+
                                   |
                    +--------------v---------------+
                    |       Data Layer              |
                    |  389 DS (LDAP) | NSS DB       |
                    +----------------------------------+

  Lightweight Protocol Responders (no LDAP backend required):
     +----------+    +---------+
     |   ACME   |    |   EST   |
     | (RFC8555)|    |(RFC7030)|
     +----------+    +---------+
```

## Subsystem Dependency Chain

```
  CA (required first)
   |--- KRA (optional, requires CA)
   |--- OCSP (optional, can be standalone)
   |--- TKS (optional, requires CA for token ops)
   |--- TPS (optional, requires CA + TKS + KRA)
   |--- ACME (lightweight, proxies to CA)
   |--- EST (lightweight, proxies to CA)
```

## Network Ports

| Port | Protocol | Purpose |
|------|----------|-------------------------------|
| 8080 | HTTP | Unsecure access (enrollment, status) |
| 8443 | HTTPS | Secure access (admin, agent, REST API) |
| 389  | LDAP | Directory Server (internal) |
| 636  | LDAPS | Secure Directory Server (internal) |

---
# 3. PKI Subsystems Reference
*Audience: All readers*

## 3.1 Certificate Authority (CA)

The CA is the **core subsystem** of Dogtag PKI. As an end user or application, you interact with the CA to request, renew, and revoke certificates.

**Capabilities:**
- X.509v3 certificate issuance and management
- Certificate Revocation List (CRL) generation and publishing
- Lightweight Sub-CAs (LWCA) for delegated authority
- 105+ built-in certificate profiles (server certs, user certs, CA certs, etc.)
- CMC (Certificate Management over CMS) protocol support
- SCEP (Simple Certificate Enrollment Protocol) support
- Security Domain management for multi-subsystem deployments
- Post-quantum cryptography: ML-DSA profile support

**REST API Endpoints (v2):**

| Servlet | Path | Purpose |
|---------|------|------|
| `CertServlet` | `/ca/v2/certs` | Certificate operations |
| `CertRequestServlet` | `/ca/v2/certrequests` | Enrollment requests |
| `AgentCertServlet` | `/ca/v2/agent/certs` | Agent certificate operations |
| `AgentCertRequestServlet` | `/ca/v2/agent/certrequests` | Agent request approval |
| `ProfileServlet` | `/ca/v2/profiles` | Profile management |
| `AuthorityServlet` | `/ca/v2/authorities` | LWCA management |
| `CAInfoServlet` | `/ca/v2/info` | System information |
| `KRAConnectorServlet` | `/ca/v2/kraconnector` | KRA connector config |
| `CAUserServlet` | `/ca/v2/admin/users` | User management |
| `CAGroupServlet` | `/ca/v2/admin/groups` | Group management |
| `CAAuditServlet` | `/ca/v2/audit` | Audit log access |
| `CASelfTestServlet` | `/ca/v2/selftests` | Self-test execution |

**Source Location:** `base/ca/`
**Artifact:** `pki-ca.jar`

---

## 3.2 Key Recovery Authority (KRA)

The KRA provides **secure key archival and recovery** services. When your organization requires key escrow -- for example, to recover encrypted emails if an employee leaves -- the KRA stores private keys in an encrypted vault.

**Capabilities:**
- Key archival (RSA, ECC) with transport-key encryption
- Key recovery with multi-agent approval
- Server-side key generation (SSKG)
- OAEP padding support for enhanced security
- CMC-based key recovery

**Source Location:** `base/kra/`
**Artifact:** `pki-kra.jar`

---

## 3.3 Online Certificate Status Protocol (OCSP) Responder

The OCSP responder provides **real-time certificate validation**. When your browser or application needs to verify whether a certificate has been revoked, it queries the OCSP responder.

**Capabilities:**
- RFC 6960 compliant OCSP responses
- CRL-based validation (direct or via LDAP)
- Standalone or CA-connected operation
- High-availability clustering

**Source Location:** `base/ocsp/`
**Artifact:** `pki-ocsp.jar`

---

## 3.4 Token Key Service (TKS)

The TKS generates **symmetric keys for smart card tokens**. It works with the TPS to perform key diversification and token personalization.

**Source Location:** `base/tks/`
**Artifact:** `pki-tks.jar`

---

## 3.5 Token Processing System (TPS)

The TPS manages the **lifecycle of smart card tokens**. It handles token enrollment, key generation, certificate loading, and PIN reset.

**Note:** The TPS module is licensed under LGPL-2.0 (all other subsystems are GPL-2.0).

**Source Location:** `base/tps/`
**Artifact:** `pki-tps.jar`

---

## 3.6 ACME Responder

The ACME responder implements **RFC 8555** for automated certificate management. If you use tools like `certbot`, you can point them at the Dogtag ACME responder for automated certificate issuance and renewal.

**Capabilities:**
- RFC 8555 Automatic Certificate Management Environment
- Pluggable database backends (in-memory, PostgreSQL, LDAP)
- Pluggable issuers (NSS, PKI)
- Pluggable realms (in-memory, DS)
- Lightweight -- no LDAP backend required

**Source Location:** `base/acme/`
**Artifact:** `pki-acme.jar`

---

## 3.7 EST Responder

The EST responder implements **RFC 7030** for Enrollment over Secure Transport. It provides a simpler alternative to CMC for certificate enrollment.

**Source Location:** `base/est/`
**Artifact:** `pki-est.jar`

---
# 4. Build System
*Audience: Developers*

## Overview

Dogtag PKI uses a **three-layer build system**:

```
  +---------------------------------------------------+
  |              build.sh (Entry Point)                |
  |  Wrapper script for developer convenience          |
  |  Targets: dist | install | src | spec | srpm | rpm |
  +---------------------------------------------------+
                         |
         +---------------+----------------+
         |                                |
  +------v-------+              +---------v--------+
  |    CMake      |              |     rpmbuild      |
  | (Native code) |              | (RPM packaging)   |
  | Orchestrates  |              | Uses pki.spec     |
  | Maven builds  |              |                   |
  +------+--------+              +------------------+
         |
  +------v--------+
  |    Maven       |
  | (Java modules) |
  | 22+ modules    |
  +----------------+
```

## Build Targets

```bash
# Developer Quick Reference

# 1. Install build dependencies (Fedora/RHEL)
sudo dnf copr -y enable @pki/master
sudo dnf builddep -y --spec pki.spec

# 2. Build from source (binaries only)
./build.sh dist
# Output: ~/build/pki/dist/*.jar + native binaries

# 3. Build RPM packages
./build.sh rpm
# Output: ~/build/pki/RPMS/*.rpm

# 4. Build with version metadata
./build.sh --with-timestamp --with-commit-id rpm

# 5. Build specific subsystems only
./build.sh --with-pkgs=base,server,ca rpm

# 6. Build with Quarkus modules
./build.sh --with-quarkus dist

# 7. Skip unit tests
./build.sh --without-test rpm

# 8. Install locally
./build.sh install
```

## Build Artifacts

After running `./build.sh dist`, the following artifacts are produced:

```
~/build/pki/dist/
  +-- pki-common.jar         # Shared libraries & client APIs
  +-- pki-tools.jar          # CLI tools
  +-- pki-tomcat.jar         # Tomcat integration (abstract)
  +-- pki-tomcat-9.0.jar     # Tomcat 9.0 specific
  +-- pki-server.jar         # Server framework
  +-- pki-server-webapp.jar  # Web application framework
  +-- pki-ca.jar             # Certificate Authority
  +-- pki-kra.jar            # Key Recovery Authority
  +-- pki-ocsp.jar           # OCSP Responder
  +-- pki-tks.jar            # Token Key Service
  +-- pki-tps.jar            # Token Processing System
  +-- pki-acme.jar           # ACME Responder
  +-- pki-est.jar            # EST Responder

Native binaries:
  +-- pistool                # PKCS#11 tool
  +-- setpin                 # PIN setting utility
  +-- tkstool               # Token key service tool
  +-- tpsclient             # Token processing client
  +-- libpki-tps.so         # TPS shared library
```

## RPM Package Map

| RPM Package | Contents |
|-------------|----------|
| `pki-base` | Shared Java libraries, client tools, Python client |
| `pki-server` | Server framework, Tomcat integration, pki-server CLI |
| `pki-ca` | Certificate Authority subsystem |
| `pki-kra` | Key Recovery Authority subsystem |
| `pki-ocsp` | OCSP Responder subsystem |
| `pki-tks` | Token Key Service subsystem |
| `pki-tps` | Token Processing System subsystem |
| `pki-acme` | ACME Responder subsystem |
| `pki-est` | EST Responder subsystem |
| `dogtag-pki` | Meta-package (installs everything) |
| `pki-javadoc` | API documentation |
| `dogtag-pki-theme` | UI themes |
| `pki-tests` | Test suites |

## Tomcat Version Detection

The build system automatically detects the appropriate Tomcat version:

| Platform | Tomcat Version |
|----------|----------------|
| Fedora < 43 | Tomcat 9.0 (javax.*) |
| Fedora >= 43 | Tomcat 10.1 (jakarta.*) |
| RHEL < 10 | Tomcat 9.0 (javax.*) |
| RHEL >= 10 | Tomcat 10.1 (jakarta.*) |

The `javax2jakarta` migration tool is applied automatically during build on platforms that require `jakarta.*` namespaces.

---
# 5. Maven Module Hierarchy
*Audience: Developers*

## Module Dependency Graph

The project contains **22 Maven modules** organized under `base/`:

```
pki-parent (org.dogtagpki.pki:pki-parent)
 +-- groupId: org.dogtagpki.pki
 +-- version: 11.10.0-SNAPSHOT
 +-- Java: 17 (source/target)
 +-- Quarkus BOM: 3.17.4
 |
 +-- base/ (pki-base-parent)
      |
      +-- CORE LIBRARIES
      |    +-- common/          (pki-common)        <-- Shared APIs & client libs
      |    +-- tools/           (pki-tools)         <-- CLI tool implementations
      |    +-- server-core/     (pki-server-core)   <-- Container-agnostic business logic
      |
      +-- TOMCAT RUNTIME
      |    +-- tomcat/          (pki-tomcat)        <-- Abstract Tomcat layer
      |    +-- tomcat-9.0/      (pki-tomcat-9.0)    <-- Tomcat 9.0 impl (javax.*)
      |    +-- tomcat-10.1/     (pki-tomcat-10.1)   <-- Tomcat 10.1 impl (jakarta.*)
      |    +-- server/          (pki-server)        <-- Tomcat-based server framework
      |    +-- server-webapp/   (pki-server-webapp) <-- Web app framework
      |
      +-- SUBSYSTEMS (Tomcat servlets)
      |    +-- ca/              (pki-ca)            <-- Certificate Authority
      |    +-- kra/             (pki-kra)           <-- Key Recovery Authority
      |    +-- ocsp/            (pki-ocsp)          <-- OCSP Responder
      |    +-- tks/             (pki-tks)           <-- Token Key Service
      |    +-- tps/             (pki-tps)           <-- Token Processing System
      |    +-- acme/            (pki-acme)          <-- ACME Responder
      |    +-- est/             (pki-est)           <-- EST Responder
      |
      +-- QUARKUS RUNTIME (New)
      |    +-- quarkus-common/  (pki-quarkus-common) <-- Shared Quarkus adapters
      |    +-- ca-quarkus/      (pki-ca-quarkus)     <-- CA on Quarkus
      |    +-- kra-quarkus/     (pki-kra-quarkus)    <-- KRA on Quarkus
      |    +-- ocsp-quarkus/    (pki-ocsp-quarkus)   <-- OCSP on Quarkus
      |    +-- tks-quarkus/     (pki-tks-quarkus)    <-- TKS on Quarkus
      |    +-- tps-quarkus/     (pki-tps-quarkus)    <-- TPS on Quarkus
      |    +-- acme-quarkus/    (pki-acme-quarkus)   <-- ACME on Quarkus
      |    +-- est-quarkus/     (pki-est-quarkus)    <-- EST on Quarkus
      |
      +-- OTHER
           +-- console/         (pki-console)       <-- Management console (optional)
```

## Key Dependency Flow

```
  pki-common  <--  pki-server-core  <--  pki-quarkus-common  <--  pki-*-quarkus
       |                |                        |
       |                |                   Quarkus BOM
       |                |                   (quarkus-arc,
       |                |                    quarkus-rest,
       |                |                    quarkus-security,
       |                |                    quarkus-vertx-http)
       |                |
       +--- pki-tomcat  +--- pki-server  <--  pki-server-webapp  <--  pki-ca/kra/...
```

The `pki-server-core` module is the **architectural pivot** -- it extracts container-agnostic business logic that can be shared between Tomcat and Quarkus deployments.

---
# 6. Source Code Structure
*Audience: Developers*

## Repository Layout

```
pki/
 +-- build.sh                 # Build entry point script
 +-- pki.spec                 # RPM spec file
 +-- pom.xml                  # Root Maven POM
 +-- CMakeLists.txt           # Root CMake config
 +-- Dockerfile               # Multi-stage container build
 +-- CLAUDE.md                # AI assistant context
 |
 +-- base/                    # All source code
 |    +-- pom.xml             # Parent POM for all modules
 |    +-- CMakeLists.txt      # CMake for base modules
 |    +-- VERSION             # Version metadata
 |    |
 |    +-- common/             # Shared Java + Python libraries
 |    |    +-- src/main/java/
 |    |    |    +-- com/netscape/certsrv/    # Public APIs
 |    |    |    +-- com/netscape/cmsutil/    # Utilities
 |    |    |    +-- org/dogtagpki/           # Modern framework
 |    |    +-- python/pki/                   # Python client lib
 |    |
 |    +-- server/             # Server framework
 |    |    +-- src/main/java/
 |    |    |    +-- com/netscape/cmscore/    # Core server impl
 |    |    |    +-- org/dogtagpki/server/    # Modern server framework
 |    |    +-- python/pki/server/            # Python management CLI
 |    |    |    +-- cli/                     # pki-server commands
 |    |    |    +-- deployment/              # pkispawn framework
 |    |    |    +-- instance.py              # Instance management
 |    |    |    +-- subsystem.py             # Subsystem management
 |    |    +-- share/conf/                   # Server config templates
 |    |    +-- share/lib/systemd/            # Systemd unit files
 |    |
 |    +-- server-core/        # Container-agnostic business logic
 |    +-- server-webapp/      # Web application framework
 |    |
 |    +-- ca/                 # Certificate Authority
 |    |    +-- src/main/java/
 |    |    |    +-- com/netscape/ca/         # CA engine
 |    |    |    +-- org/dogtagpki/server/ca/ # Modern CA framework
 |    |    |         +-- rest/v2/            # REST API v2 servlets
 |    |    +-- shared/
 |    |    |    +-- conf/                    # ACLs, auth config
 |    |    |    +-- profiles/ca/             # 105 certificate profiles
 |    |    |    +-- webapps/                 # Web UI resources
 |    |
 |    +-- kra/                # Key Recovery Authority
 |    +-- ocsp/               # OCSP Responder
 |    +-- tks/                # Token Key Service
 |    +-- tps/                # Token Processing System
 |    +-- acme/               # ACME Responder
 |    +-- est/                # EST Responder
 |    |
 |    +-- quarkus-common/     # Shared Quarkus adapters
 |    +-- ca-quarkus/         # CA Quarkus deployment module
 |    +-- kra-quarkus/        # KRA Quarkus deployment module
 |    +-- ocsp-quarkus/       # OCSP Quarkus deployment module
 |    +-- tks-quarkus/        # TKS Quarkus deployment module
 |    +-- tps-quarkus/        # TPS Quarkus deployment module
 |    +-- acme-quarkus/       # ACME Quarkus deployment module
 |    +-- est-quarkus/        # EST Quarkus deployment module
 |    |
 |    +-- tomcat/             # Abstract Tomcat layer
 |    +-- tomcat-9.0/         # Tomcat 9.0 implementation
 |    +-- tomcat-10.1/        # Tomcat 10.1 implementation
 |    +-- tools/              # Native C utilities + CLI tools
 |    +-- console/            # Management console
 |
 +-- cmake/                   # CMake modules
 +-- docs/                    # Documentation
 |    +-- installation/       # Deployment guides
 |    +-- development/        # Developer docs
 |    +-- admin/              # Admin guides
 |    +-- user/               # User guides
 |    +-- manuals/            # Man pages
 |
 +-- tests/                   # Integration tests
 |    +-- bin/                # Test infrastructure scripts
 |    +-- pylintrc            # Python lint config
 |
 +-- themes/                  # UI themes (dogtag)
 +-- .github/workflows/       # 187 CI/CD workflow files
```

## Java Package Map

| Package | Purpose | Layer |
|---------|---------|-------|
| `com.netscape.certsrv.*` | Public APIs and client libraries | Common |
| `com.netscape.cmsutil.*` | Cryptographic and encoding utilities | Common |
| `com.netscape.cmscore.*` | Server core implementation | Server |
| `com.netscape.ca.*` | CA engine internals | CA |
| `org.dogtagpki.*` | Modern framework (migration target) | All |
| `org.dogtagpki.server.ca.rest.v2.*` | CA REST API v2 (servlets + filters) | CA |

## Codebase Statistics

| Metric | Count |
|--------|-------|
| Java source files | 3,494 |
| Java lines of code | ~683,000 |
| Python source files | 189 |
| Python lines of code | ~60,000 |
| C/C++ source files | 109 |
| Certificate profiles | 105 |
| CI/CD workflows | 187 |
| Maven modules | 22 |

---
# 7. Deployment Guide
*Audience: System Administrators*

## Prerequisites

Before deploying Dogtag PKI, ensure the following are available on your system:

1. **Operating System:** Fedora (latest) or RHEL 9+
2. **389 Directory Server:** For LDAP backend
3. **Java Runtime:** OpenJDK 17, 21, or 25 (platform-dependent)
4. **NSS:** Mozilla Network Security Services with FIPS support

## Installation from RPM

```bash
# Install everything (meta-package)
sudo dnf install dogtag-pki

# Or install specific subsystems
sudo dnf install pki-ca              # Certificate Authority only
sudo dnf install pki-ca pki-kra      # CA + Key Recovery Authority
```

## Deploying a CA Instance

### Step 1: Create the PKI Server Instance

```bash
# Create a new server instance
pki-server create pki-tomcat
```

### Step 2: Deploy Using pkispawn

```bash
# Create a configuration file (ca.cfg)
cat > /tmp/ca.cfg << 'EOF'
[DEFAULT]
pki_admin_password=Secret.123
pki_client_pkcs12_password=Secret.123
pki_ds_password=Secret.123

[CA]
pki_admin_email=caadmin@example.com
pki_admin_name=caadmin
pki_admin_nickname=PKI CA Administrator
pki_ds_hostname=localhost
pki_ds_ldap_port=389
pki_ds_base_dn=dc=ca,dc=pki,dc=example,dc=com
EOF

# Deploy the CA
pkispawn -f /tmp/ca.cfg -s CA
```

### Step 3: Verify the Deployment

```bash
# Check instance status
pki-server status

# Verify the CA is responding
curl -k https://localhost:8443/ca/v2/info
```

### Step 4: Add Additional Subsystems

```bash
# Add KRA to existing CA instance
pkispawn -f /tmp/kra.cfg -s KRA

# Add OCSP responder
pkispawn -f /tmp/ocsp.cfg -s OCSP
```

## Destroying an Instance

```bash
# Remove a specific subsystem
pkidestroy -i pki-tomcat -s CA

# Remove the entire instance
pki-server remove pki-tomcat
```

## Instance File Layout

After deployment, the instance is stored at:

```
/var/lib/pki/pki-tomcat/
  +-- conf/                # Instance configuration
  |    +-- ca/             # CA subsystem config (CS.cfg)
  |    +-- server.xml      # Tomcat server config
  |    +-- password.conf   # NSS database passwords
  +-- logs/                # Instance logs
  +-- alias/               # NSS database (certs & keys)
  +-- webapps/             # Deployed web applications

/etc/pki/pki-tomcat/       # Symlink to conf/
/var/log/pki/pki-tomcat/   # Symlink to logs/
```

---
# 8. Container Deployment
*Audience: System Administrators, DevOps Engineers*

## Dockerfile Architecture

The project ships a **multi-stage Dockerfile** that produces specialized container images for each subsystem:

```
  Build Stage Hierarchy:
  =====================

  BASE_IMAGE (fedora:latest)
       |
       v
  pki-base                     # Base: dnf, systemd
       |
       +----> pki-deps         # Runtime dependencies installed
       |         |
       |         +----> pki-builder-deps    # Build dependencies
       |         |         |
       |         |         +----> pki-builder          # Tomcat RPM build
       |         |         |         |
       |         |         |         v
       |         |         |      pki-dist             # RPM artifacts (alpine)
       |         |         |
       |         |         +----> pki-quarkus-builder  # Quarkus Maven build
       |         |
       |         +----> pki-runner         # Tomcat runtime
       |         |         |
       |         |         +----> pki-server   # Base server image
       |         |                   |
       |         |                   +----> pki-ca      # CA image
       |         |                   +----> pki-kra     # KRA image
       |         |                   +----> pki-ocsp    # OCSP image
       |         |                   +----> pki-tks     # TKS image
       |         |                   +----> pki-tps     # TPS image
       |         |                   +----> pki-acme    # ACME image
       |         |
       |         +----> ipa-runner         # FreeIPA integration
       |
       +----> pki-quarkus-runner   # Quarkus runtime (minimal)
                  |
                  +----> pki-quarkus-ca     # CA on Quarkus
                  +----> pki-quarkus-est    # EST on Quarkus
                  +----> pki-quarkus-acme   # ACME on Quarkus
                  +----> pki-quarkus-ocsp   # OCSP on Quarkus
                  +----> pki-quarkus-kra    # KRA on Quarkus
                  +----> pki-quarkus-tks    # TKS on Quarkus
                  +----> pki-quarkus-tps    # TPS on Quarkus
```

## Building Container Images

```bash
# Build a specific subsystem image (Tomcat-based)
podman build --target pki-ca -t pki-ca .

# Build a Quarkus-based subsystem image
podman build --target pki-quarkus-ca -t pki-quarkus-ca .

# Build with a specific COPR repo for dependencies
podman build --target pki-ca \
    --build-arg COPR_REPO=@pki/master \
    -t pki-ca .
```

## Running Container Images

### Tomcat-Based CA

```bash
podman run -d \
    --name pki-ca \
    -p 8080:8080 \
    -p 8443:8443 \
    -v pki-certs:/certs \
    -v pki-conf:/conf \
    -v pki-logs:/logs \
    pki-ca
```

### Quarkus-Based CA

```bash
podman run -d \
    --name pki-quarkus-ca \
    -p 8080:8080 \
    -p 8443:8443 \
    pki-quarkus-ca
# Runs: java -jar /usr/share/pki/quarkus/ca/quarkus-run.jar
```

## Container Volumes

| Volume | Purpose |
|--------|------|
| `/certs` | NSS database and certificates |
| `/conf` | Instance configuration files |
| `/logs` | Server and audit logs |
| `/metadata` | ACME metadata (ACME only) |
| `/database` | ACME database config (ACME only) |
| `/issuer` | ACME issuer config (ACME only) |
| `/realm` | ACME realm config (ACME only) |

## OpenShift Compatibility

The container images are designed for OpenShift deployment:
- Server runs as `pkiuser` (UID 17) with `root` group (GID 0)
- Supports arbitrary UID assignment by OpenShift
- File permissions set with `g+rw` for root group access

---
# 9. Certificate Profiles
*Audience: End Users, Security Administrators*

## What is a Certificate Profile?

A certificate profile defines the **template** for a specific type of certificate. When you request a certificate, you select a profile that determines:
- What information you need to provide (subject name, extensions)
- How the request is authenticated
- What X.509 extensions are included
- The validity period and key usage constraints

## Built-in Profiles (105 total)

### Server Certificates
| Profile | Description |
|---------|------|
| `caServerCert` | Standard TLS server certificate |
| `caServerCertWithSCT` | Server cert with Signed Certificate Timestamp |
| `caServerCertWithCRLDP` | Server cert with CRL Distribution Point |
| `caECServerCert` | ECC-based TLS server certificate |
| `caMLDSAServerCert` | Post-quantum ML-DSA server certificate |
| `acmeServerCert` | ACME-issued server certificate |
| `caAgentServerCert` | Server cert for PKI agent authentication |
| `caIPAserviceCert` | FreeIPA service certificate |

### User Certificates
| Profile | Description |
|---------|------|
| `caUserCert` | Standard user certificate (RSA) |
| `caECUserCert` | ECC-based user certificate |
| `caMLDSAUserCert` | Post-quantum ML-DSA user certificate |
| `caDirUserCert` | Directory-authenticated user cert |
| `caDualCert` | Dual-key (signing + encryption) user cert |
| `caEncUserCert` | Encryption-only user certificate |
| `caSigningUserCert` | Signing-only user certificate |

### CA and Infrastructure Certificates
| Profile | Description |
|---------|------|
| `caCACert` | CA signing certificate |
| `caCrossSignedCACert` | Cross-signed CA certificate |
| `caSubsystemCert` | PKI subsystem certificate |
| `caOCSPCert` | OCSP responder signing certificate |
| `caAuditSigningCert` | Audit log signing certificate |
| `caStorageCert` | KRA storage certificate |
| `caTransportCert` | KRA transport certificate |

### Token (Smart Card) Certificates
| Profile | Description |
|---------|------|
| `caTokenDeviceKeyEnrollment` | Token device key certificate |
| `caTokenUserSigningKeyEnrollment` | Token user signing key |
| `caTokenUserEncryptionKeyEnrollment` | Token user encryption key |
| `caTokenMSLoginEnrollment` | Token for MS login |

### CMC (Certificate Management over CMS) Profiles
| Profile | Description |
|---------|------|
| `caCMCUserCert` | CMC-enrolled user certificate |
| `caCMCserverCert` | CMC-enrolled server certificate |
| `caFullCMCUserCert` | Full CMC user enrollment |
| `caSimpleCMCUserCert` | Simple CMC user enrollment |
| `caFullCMCSharedTokenCert` | CMC with shared token |

### Post-Quantum Cryptography
| Profile | Description |
|---------|------|
| `caMLDSAServerCert` | ML-DSA (FIPS 204) server certificate |
| `caMLDSAUserCert` | ML-DSA user certificate |
| `caMLDSAAdminCert` | ML-DSA admin certificate |
| `caMLDSASubsystemCert` | ML-DSA subsystem certificate |
| `caMLDSAInternalAuthServerCert` | ML-DSA internal auth server cert |

## Profile Configuration Format

Profiles are stored as `.cfg` files in `base/ca/shared/profiles/ca/`. Each profile defines:

```properties
# Profile identity
profileId=caServerCert
classId=caEnrollImpl
name=Manual Server Certificate Enrollment

# Authentication (who can request)
auth.instance_id=AgentCertAuth

# Input (what the requester provides)
input.list=i1
input.i1.class_id=certReqInputImpl

# Policy (what constraints are enforced)
policyset.list=serverCertSet
policyset.serverCertSet.list=1,2,3,4,...
policyset.serverCertSet.1.constraint.class_id=...
policyset.serverCertSet.1.default.class_id=...

# Output (what is returned)
output.list=o1
output.o1.class_id=certOutputImpl
```

---
# 10. Server Management
*Audience: System Administrators*

## The pki-server CLI

The `pki-server` command is a Python-based management tool for PKI server instances. As an administrator, you use this to create, configure, and manage PKI deployments.

### Instance Lifecycle

```bash
# Create a new PKI server instance
pki-server create <instance-id>

# Start/stop/restart the instance
systemctl start pki-tomcatd@<instance-id>
systemctl stop pki-tomcatd@<instance-id>
systemctl restart pki-tomcatd@<instance-id>

# Check instance status
pki-server status

# Remove an instance
pki-server remove <instance-id>
```

### NSS Database Management

```bash
# Create NSS database
pki-server nss-create --no-password

# Enable JSS (Java Security Services)
pki-server jss-enable
```

### HTTP Connector Configuration

```bash
# Add an HTTPS connector
pki-server http-connector-add \
    --port 8443 \
    --scheme https \
    --secure true \
    --sslEnabled true \
    --sslProtocol SSL \
    --sslImpl org.dogtagpki.jss.tomcat.JSSImplementation \
    Secure

# Add SSL server certificate
pki-server http-connector-cert-add \
    --keyAlias sslserver \
    --keystoreType pkcs11 \
    --keystoreProvider Mozilla-JSS
```

### Web Application Deployment

```bash
# Deploy ROOT webapp
pki-server webapp-deploy \
    --descriptor /usr/share/pki/server/conf/Catalina/localhost/ROOT.xml \
    ROOT

# Deploy PKI webapp
pki-server webapp-deploy \
    --descriptor /usr/share/pki/server/conf/Catalina/localhost/pki.xml \
    pki
```

### Subsystem Management

```bash
# Create and deploy a CA subsystem
pki-server ca-create
pki-server ca-deploy

# Create and deploy a KRA subsystem
pki-server kra-create
pki-server kra-deploy

# Create and deploy ACME responder
pki-server acme-create
pki-server acme-deploy
```

### Systemd Integration

```
Tomcat-based services:
  pki-tomcatd@<instance>.service   # Per-instance Tomcat service

Quarkus-based services (new):
  pki-quarkusd@<subsystem>.service # Per-subsystem Quarkus service
  pki-quarkusd.target              # Target for all Quarkus subsystems
```

---
# 11. Tomcat-to-Quarkus Migration
*Audience: Developers, System Administrators*

## Why Quarkus?

The project is migrating from Apache Tomcat to **Quarkus 3.17.4** to achieve:

| Benefit | Details |
|---------|------|
| **Faster startup** | Quarkus boots in seconds vs. Tomcat's longer initialization |
| **Lower memory** | Reduced JVM heap and metaspace usage |
| **Cloud-native** | First-class Kubernetes/OpenShift support |
| **Modern stack** | CDI, Vert.x, RESTEasy Reactive |
| **Native compilation** | GraalVM native-image support (future) |
| **Per-subsystem isolation** | Each subsystem runs as its own process |

## Architecture Change

```
  BEFORE (Tomcat):                       AFTER (Quarkus):
  ================                       ================

  Single Tomcat JVM                      Separate JVM per subsystem
  +-------------------+                  +--------+  +--------+
  | Tomcat Instance    |                  | CA     |  | KRA    |
  |  +-- CA webapp     |                  | (JVM)  |  | (JVM)  |
  |  +-- KRA webapp    |    =======>      +--------+  +--------+
  |  +-- OCSP webapp   |                  +--------+  +--------+
  |  +-- TKS webapp    |                  | OCSP   |  | TKS    |
  |  +-- TPS webapp    |                  | (JVM)  |  | (JVM)  |
  +-------------------+                  +--------+  +--------+
  pki-tomcatd@inst.svc                   pki-quarkusd@ca.svc
                                         pki-quarkusd@kra.svc
                                         pki-quarkusd@ocsp.svc
                                         ...
```

## Module Strategy

The migration introduces three new architectural layers:

### 1. `pki-server-core` (Container-Agnostic Business Logic)
Extracted from `pki-server` -- contains all business logic that works independently of Tomcat or Quarkus.

```
pki-common --> pki-server-core --> pki-quarkus-common --> pki-ca-quarkus
                                                      --> pki-kra-quarkus
                                                      --> ...
```

### 2. `pki-quarkus-common` (Shared Quarkus Adapters)
Provides Quarkus-specific implementations:
- CDI producers for PKI services
- Vert.x HTTP handlers for client certificate auth
- Quarkus security integration
- Configuration bridge (CS.cfg to Quarkus config)

### 3. `pki-*-quarkus` (Per-Subsystem Quarkus Apps)
Each subsystem has its own Quarkus deployment module that:
- Bundles the subsystem JAR + server-core + quarkus-common
- Provides `application.properties` / `application.yaml`
- Produces a `quarkus-app/` directory with runnable JAR

## Quarkus Dependencies per Module

```xml
<!-- Core Quarkus stack used by each subsystem -->
quarkus-arc              <!-- CDI container -->
quarkus-rest             <!-- RESTEasy Reactive (JAX-RS) -->
quarkus-rest-jackson     <!-- JSON serialization -->
quarkus-security         <!-- Security framework -->
quarkus-vertx-http       <!-- Vert.x HTTP (client certs) -->
quarkus-config-yaml      <!-- YAML configuration -->
quarkus-logging-json     <!-- Structured logging -->
```

## Running Quarkus Subsystems

```bash
# Direct execution
java -jar /usr/share/pki/quarkus/ca/quarkus-run.jar

# Via systemd
systemctl start pki-quarkusd@ca
systemctl start pki-quarkusd@kra

# Via container
podman run -p 8080:8080 -p 8443:8443 pki-quarkus-ca
```

## Build Pipeline (Parallel)

The Tomcat and Quarkus builds are **decoupled** and can run in parallel:

```
  pki-builder-deps
       |
       +-----> pki-builder         (rpmbuild, Tomcat RPMs)
       |           |                   |
       |           v                   v
       |       pki-dist           pki-runner --> pki-ca, pki-kra, ...
       |
       +-----> pki-quarkus-builder (mvn install, Quarkus JARs)  [PARALLEL]
                   |
                   v
               pki-quarkus-runner --> pki-quarkus-ca, pki-quarkus-kra, ...
```

## javax to jakarta Migration

On platforms using Tomcat 10.1+ (Fedora 43+, RHEL 10+), the source code is automatically migrated from `javax.*` to `jakarta.*` namespaces during build using the `javax2jakarta` tool. The Quarkus modules always use `jakarta.*`.

---
# 12. Access Control & Authorization
*Audience: Security Administrators, Auditors*

## RBAC Model

Dogtag PKI implements a **role-based access control (RBAC)** system with the following roles:

| Role | Description | Typical Operations |
|------|------|------|
| **Administrator** | Full system management | User/group management, configuration |
| **Agent** | Certificate operations | Approve/reject/revoke certificates |
| **Auditor** | Audit log access | View and export signed audit logs |
| **End Entity** | Self-service | Request certificates, check status |

## ACL Configuration

Access control is defined in `acl.properties` files per subsystem. Each entry maps an operation to a resource and permission:

```properties
# Format: <mapping-name> = <resource-ID>,<operation>

# Account operations
account.login = certServer.ca.account,login
account.logout = certServer.ca.account,logout

# Certificate operations (Agent role required)
certs = certServer.ca.certs,execute
certrequests = certServer.ca.certrequests,execute

# Audit configuration (Auditor/Admin role)
audit.read = certServer.log.configuration,read
audit.modify = certServer.log.configuration,modify
audit-log.read = certServer.log.content.signedAudit,read

# Profile management (Admin role)
profiles.approve = certServer.ca.profile,approve
profiles.create = certServer.profile.configuration,modify
profiles.delete = certServer.profile.configuration,modify
profiles.list = certServer.ee.profiles,list
profiles.modify = certServer.profile.configuration,modify
profiles.read = certServer.profile.configuration,read

# User management (Admin role)
users = certServer.ca.users,execute
groups = certServer.ca.groups,execute

# Security Domain (Admin role)
securityDomain.read = certServer.securitydomain.domainxml,read
securityDomain.modify = certServer.securitydomain.domainxml,modify

# Self-tests (Admin role)
selftests.read = certServer.ca.selftests,read
selftests.execute = certServer.ca.selftests,execute

# KRA Connector (Admin role)
kraconnectors = certServer.ca.connectorInfo,modify
```

## Authentication Methods

Each REST endpoint has an **authentication method** and **ACL filter** enforced via servlet filters:

```
  HTTP Request
      |
      v
  AuthMethod Filter  ---> Validates client certificate / credentials
      |
      v
  ACL Filter         ---> Checks role-based permissions
      |
      v
  Servlet Handler    ---> Processes the request
```

The CA subsystem, for example, has dedicated filter classes:
- `DashboardAuthMethod` - Authentication filter for dashboard endpoints
- `DashboardACL` - Authorization filter for dashboard endpoints

## Client Authentication

PKI uses **mutual TLS (mTLS)** as the primary authentication mechanism:
- Agents authenticate with agent certificates stored in NSS databases
- Admins authenticate with admin certificates
- Subsystem-to-subsystem communication uses subsystem certificates
- End entities may use username/password or directory-based auth

---
# 13. Security Architecture
*Audience: Security Administrators, Auditors*

## Cryptographic Foundation

All cryptographic operations in Dogtag PKI are performed by **Mozilla NSS (Network Security Services)**, a FIPS 140-2 validated cryptographic library.

```
  +-----------------------------------------------------------+
  |                    Application Layer                       |
  |  (Java / Python / C)                                      |
  +-----------------------------------------------------------+
  |                    JSS (Java Security Services)            |
  |  Java wrapper for NSS via JNI                              |
  +-----------------------------------------------------------+
  |                    Mozilla NSS                              |
  |  FIPS 140-2 validated crypto library                       |
  |  Algorithms: RSA, ECC, ML-DSA, AES, SHA-2/3               |
  +-----------------------------------------------------------+
  |                    PKCS#11 Interface                        |
  |  Software token or HSM (e.g., SafeNet Luna, Thales)        |
  +-----------------------------------------------------------+
```

## Supported Algorithms

| Category | Algorithms |
|----------|------|
| **Asymmetric** | RSA (2048-8192), ECDSA (P-256, P-384, P-521), ML-DSA (post-quantum) |
| **Symmetric** | AES-128/256 (CBC, GCM) |
| **Hash** | SHA-256, SHA-384, SHA-512, SHA-3 |
| **Key Exchange** | ECDH, RSA, RSA-OAEP |
| **TLS** | TLS 1.2, TLS 1.3 |

## HSM (Hardware Security Module) Support

Dogtag PKI supports **Hardware Security Modules** for protecting CA signing keys and KRA storage keys:

- PKCS#11 interface for any compliant HSM
- Tested with SafeNet Luna, Thales nShield
- Dedicated CI/CD tests for HSM integration
- Supports FIPS 140-2 Level 3 operation

## NSS Database

Each PKI instance maintains an **NSS database** (`alias/` directory) containing:

| Certificate | Purpose |
|-------------|------|
| CA Signing Cert | Signs issued certificates and CRLs |
| OCSP Signing Cert | Signs OCSP responses |
| SSL Server Cert | TLS server identity |
| Subsystem Cert | Subsystem-to-subsystem authentication |
| Audit Signing Cert | Signs audit log entries |
| KRA Storage Cert | Encrypts archived private keys |
| KRA Transport Cert | Encrypts keys in transit to KRA |

## Security Policies

The server enforces security through Java Security Manager policies:

| Policy File | Purpose |
|-------------|------|
| `catalina.policy` | Base Tomcat security policy |
| `pki.policy` | PKI-specific security permissions |
| `custom.policy` | Site-specific customizations |

## Data Protection

| Data Type | Protection Mechanism |
|-----------|------|
| CA private key | NSS database (software) or HSM (hardware) |
| Archived user keys | Triple-wrapped: transport key + storage key + session key |
| Audit logs | Digitally signed with audit signing certificate |
| LDAP passwords | Stored encrypted in `password.conf` |
| Configuration secrets | Protected by file permissions (pkiuser:pkiuser) |
| TLS sessions | NSS-managed SSL context with FIPS-approved ciphersuites |

---
# 14. Audit & Compliance
*Audience: Auditors, Security Administrators*

## Audit Logging

Dogtag PKI maintains **digitally signed audit logs** for all security-relevant operations. As an auditor, you can verify the integrity of these logs using the audit signing certificate.

### Audited Events

| Event Category | Examples |
|---------------|------|
| **Authentication** | Login, logout, failed authentication attempts |
| **Certificate Lifecycle** | Issuance, renewal, revocation, hold, release |
| **Key Management** | Key archival, key recovery, key generation |
| **Configuration** | Profile changes, ACL modifications, connector setup |
| **Access Control** | Authorization successes and failures |
| **Self-Tests** | Self-test execution and results |
| **CRL Operations** | CRL generation, publishing, update scheduling |

### Accessing Audit Logs

```bash
# Via REST API (requires Auditor role)
curl -k --cert auditor.pem \
    https://localhost:8443/ca/v2/audit

# Via CLI
pki -d ~/.dogtag/nssdb -c password \
    -n "PKI Auditor" \
    ca-audit-file-find
```

### Log Integrity Verification

Each audit log entry is signed with the instance's audit signing certificate. You can verify integrity by:
1. Extracting the audit signing certificate from the NSS database
2. Validating signatures on each log segment
3. Checking for gaps or modifications in the signed sequence

## Compliance Standards

Dogtag PKI is designed to support compliance with:

| Standard | Relevance |
|----------|------|
| **Common Criteria** | Evaluated at EAL4+ for CIMC Protection Profile |
| **FIPS 140-2** | NSS cryptographic module (validated) |
| **RFC 5280** | X.509 certificate and CRL profile |
| **RFC 6960** | OCSP response generation |
| **RFC 8555** | ACME protocol compliance |
| **RFC 7030** | EST protocol compliance |
| **WebTrust** | CA/Browser Forum baseline requirements support |

## Self-Tests

The system includes built-in self-tests that verify:
- Cryptographic operations (signing, verification)
- Certificate chain validation
- LDAP connectivity
- NSS database integrity

```bash
# Execute self-tests via REST API
curl -k --cert admin.pem \
    -X POST \
    https://localhost:8443/ca/v2/selftests/run

# View self-test results
curl -k --cert admin.pem \
    https://localhost:8443/ca/v2/selftests
```

---
# 15. CI/CD & Testing
*Audience: Developers, Auditors*

## Testing Infrastructure

Dogtag PKI has **187 GitHub Actions workflow files** providing comprehensive automated testing.

## Test Categories

```
  CI/CD Test Categories (187 workflows)
  ======================================

  Basic Functionality:
    +-- pki-basic-test           # Core PKI operations
    +-- pki-server-basic-test    # Server lifecycle
    +-- pki-password-test        # Password management

  CA Tests:
    +-- ca-tests, ca-tests2      # Certificate operations
    +-- ca-crl-test              # CRL generation
    +-- ca-profile-custom-test   # Custom profiles
    +-- ca-clone-*               # CA cloning & replication
    +-- ca-renewal-*             # Certificate renewal
    +-- ca-container-*           # Container deployment
    +-- ca-ssnv1/v2-test         # Serial number versions
    +-- lwca-*                   # Lightweight sub-CA

  KRA Tests:
    +-- kra-tests                # Key archival & recovery
    +-- kra-clone-*              # KRA cloning
    +-- kra-cmc-test             # CMC key recovery
    +-- kra-ecc-test             # ECC key support
    +-- kra-oaep-test            # OAEP padding
    +-- kra-standalone-test      # Standalone KRA
    +-- kra-sskg-test            # Server-side key gen
    +-- kra-hsm-test             # HSM integration

  OCSP Tests:
    +-- ocsp-tests               # OCSP responses
    +-- ocsp-crl-direct-test     # Direct CRL validation
    +-- ocsp-crl-ldap-test       # LDAP-based CRL
    +-- ocsp-standalone-test     # Standalone OCSP

  ACME Tests:
    +-- acme-container-*         # Container ACME
    +-- acme-postgresql-test     # PostgreSQL backend
    +-- acme-clone-test          # ACME cloning
    +-- acme-switchover-test     # Failover testing

  EST Tests:
    +-- est-basic-test           # Basic enrollment
    +-- est-standalone-test      # Standalone EST
    +-- est-ds-realm-*           # Directory-backed realm

  Protocol Tests:
    +-- pki-nss-*                # NSS operations (RSA, ECC, AES)
    +-- pki-pkcs7/11/12-test     # PKCS format handling
    +-- scep-test                # SCEP protocol
    +-- PKCS10Client-test        # PKCS#10 requests

  Security Tests:
    +-- server-https-*           # TLS with JKS/NSS/PEM/PKCS12
    +-- *-hsm-test               # HSM integration

  Integration Tests:
    +-- ipa-*                    # FreeIPA integration
    +-- python-*                 # Python client tests

  Code Quality:
    +-- code-analysis            # Static analysis
    +-- python-lint-test         # Python linting
    +-- rpminspect-test          # RPM compliance
```

## Running Tests

### Unit Tests

```bash
# Run unit tests during build (enabled by default)
./build.sh dist

# Skip unit tests
./build.sh --without-test rpm
```

### Python Linting

```bash
# Lint Python code
pylint --rcfile=tests/pylintrc <file.py>
flake8 <file.py>
```

### Integration Tests

Integration tests are container-based and run via GitHub Actions. Test infrastructure scripts in `tests/bin/` handle:
- Container creation and management
- 389 Directory Server setup
- PKI instance deployment
- Certificate operations validation
- Cleanup and teardown

---
# 16. REST API Quick Reference
*Audience: Developers, End Users*

## Authentication

Most API endpoints require **client certificate authentication** via mutual TLS.

```bash
# Using curl with client certificate
curl -k \
    --cert /path/to/client.pem \
    --key /path/to/client-key.pem \
    https://localhost:8443/ca/v2/info

# Using PKI CLI with NSS database
pki -d ~/.dogtag/nssdb -c <password> \
    -n "PKI Administrator" \
    ca-cert-find
```

## Common API Operations

### Get System Info
```
GET /ca/v2/info
```

### Search Certificates
```
GET /ca/v2/certs
GET /ca/v2/certs/{id}
```

### Submit Certificate Request
```
POST /ca/v2/certrequests
Content-Type: application/json

{
  "ProfileID": "caServerCert",
  "Input": [
    {
      "id": "i1",
      "ClassID": "certReqInputImpl",
      "Attribute": [
        {
          "name": "cert_request",
          "value": "<PKCS10 CSR>"
        }
      ]
    }
  ]
}
```

### Approve Certificate Request (Agent)
```
POST /ca/v2/agent/certrequests/{id}/approve
```

### Revoke Certificate (Agent)
```
POST /ca/v2/agent/certs/{id}/revoke
```

### List Profiles
```
GET /ca/v2/profiles
GET /ca/v2/profiles/{id}
```

### Manage Users (Admin)
```
GET    /ca/v2/admin/users
POST   /ca/v2/admin/users
GET    /ca/v2/admin/users/{id}
DELETE /ca/v2/admin/users/{id}
```

### Manage Groups (Admin)
```
GET    /ca/v2/admin/groups
POST   /ca/v2/admin/groups
GET    /ca/v2/admin/groups/{id}
DELETE /ca/v2/admin/groups/{id}
```

### Audit Operations (Auditor)
```
GET /ca/v2/audit
GET /ca/v2/audit/config
```

---
# 17. Versioning & Release Strategy
*Audience: Developers, System Administrators*

## Version Format

```
  <major>.<minor>.<update>[-<phase>]

  Current: 11.10.0-alpha1
            |   |   |  |
            |   |   |  +-- Development phase
            |   |   +-- Update (patch) version
            |   +-- Minor version
            +-- Major version
```

## Development Phases

| Phase | Status | RPM Version Format |
|-------|--------|------|
| `alpha<n>` | Development (unsupported) | `11.10.0~alpha1^20260217.abcd1234` |
| `beta<n>` | Stabilization (supported) | `11.10.0~beta1` |
| *(none)* | GA/Update (supported) | `11.10.0` |

## Branch Naming

| Branch Pattern | Purpose |
|---------------|------|
| `master` | Main development branch |
| `v<major>` | Major version branch |
| `v<major>.<minor>` | Minor version branch |
| `DOGTAG_<major>_<minor>_BRANCH` | Legacy branch format |

## Java Version Strategy

| Platform | Java Version |
|----------|------|
| RHEL < 9 | Java 17 |
| Fedora < 43, RHEL 9+ | Java 21 |
| Fedora 43+ | Java 25 |

---
# 18. Quick Reference Card
*Audience: All readers*

## Essential Commands

### For Developers
```bash
# Build from source
./build.sh dist

# Build RPMs
./build.sh rpm

# Build with Quarkus
./build.sh --with-quarkus dist

# Build specific subsystems
./build.sh --with-pkgs=base,server,ca rpm

# Lint Python code
pylint --rcfile=tests/pylintrc <file.py>
```

### For System Administrators
```bash
# Install PKI
sudo dnf install dogtag-pki

# Deploy CA
pkispawn -f ca.cfg -s CA

# Manage instance
pki-server status
systemctl start pki-tomcatd@pki-tomcat
systemctl stop pki-tomcatd@pki-tomcat

# Remove subsystem
pkidestroy -i pki-tomcat -s CA
```

### For End Users
```bash
# Request a certificate
pki -d ~/.dogtag/nssdb -c <password> \
    client-cert-request uid=testuser

# Check certificate status
pki ca-cert-show <serial-number>

# List available profiles
pki ca-profile-find
```

### For Container Users
```bash
# Build CA container (Tomcat)
podman build --target pki-ca -t pki-ca .

# Build CA container (Quarkus)
podman build --target pki-quarkus-ca -t pki-quarkus-ca .

# Run CA container
podman run -p 8080:8080 -p 8443:8443 pki-ca
```

## Key File Locations

| File | Purpose |
|------|------|
| `build.sh` | Build entry point |
| `pki.spec` | RPM spec file |
| `pom.xml` | Root Maven POM |
| `CMakeLists.txt` | Root CMake config |
| `Dockerfile` | Multi-stage container build |
| `base/pom.xml` | Module hierarchy definition |
| `base/ca/shared/conf/acl.properties` | CA access control |
| `base/ca/shared/profiles/ca/` | Certificate profiles (105 files) |
| `base/server/python/pki/server/` | Python management CLI |
| `.github/workflows/` | CI/CD definitions (187 files) |

## Useful Links

| Resource | URL |
|----------|------|
| Source Code | `https://github.com/dogtagpki/pki` |
| Wiki | `https://github.com/dogtagpki/pki/wiki` |
| Maven Repo | `https://raw.githubusercontent.com/dogtagpki/repo/maven` |
| Container Images | `quay.io/dogtagpki/` |

---
# 19. Glossary
*Audience: All readers*

| Term | Definition |
|------|------|
| **ACME** | Automatic Certificate Management Environment (RFC 8555) |
| **CA** | Certificate Authority -- issues and manages digital certificates |
| **CDI** | Contexts and Dependency Injection -- Java EE/Jakarta standard |
| **CMC** | Certificate Management over CMS -- advanced enrollment protocol |
| **CRL** | Certificate Revocation List -- list of revoked certificates |
| **EST** | Enrollment over Secure Transport (RFC 7030) |
| **HSM** | Hardware Security Module -- tamper-resistant crypto device |
| **JSS** | Java Security Services -- Java binding for NSS |
| **KRA** | Key Recovery Authority -- archives and recovers private keys |
| **LDAP** | Lightweight Directory Access Protocol |
| **LWCA** | Lightweight Certificate Authority (sub-CA) |
| **ML-DSA** | Module-Lattice Digital Signature Algorithm (post-quantum, FIPS 204) |
| **mTLS** | Mutual TLS -- both client and server present certificates |
| **NSS** | Network Security Services -- Mozilla's crypto library |
| **OCSP** | Online Certificate Status Protocol (RFC 6960) |
| **PKCS#11** | Cryptographic Token Interface Standard |
| **RBAC** | Role-Based Access Control |
| **SCEP** | Simple Certificate Enrollment Protocol |
| **TKS** | Token Key Service -- generates symmetric keys for smart cards |
| **TPS** | Token Processing System -- manages smart card lifecycle |

---
*This document was generated from the Dogtag PKI source code repository on branch `tomcat-to-quarkus-migration`. It reflects the system state as of February 2026, version 11.10.0-alpha1.*